# Visualization of the paths obtained, stored in csv files that were saved after executing queries

In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
conn = psycopg2.connect(
    host="localhost",
    database="krakow_90",
    user="postgres",
    password="postgres")

conn.autocommit = True
cursor = conn.cursor() 

In [11]:
def set_paths(mode, length):
    original_input_path = f"C:/Users/kisie/Documents/nova-dm/query_results/dijkstra_{mode}_{length}.csv"
    transformed_input_path = f"C:/Users/kisie/Documents/nova-dm/{mode}_{length}.csv"
    return original_input_path, transformed_input_path


In [12]:
def transform_csv_file(input_path, output_path):
    df = pd.read_csv(input_path)
    df['source'] = df['node']
    df['target'] = df['node'].shift(periods=-1)
    df = df.dropna()
    df['target'] = df['target'].astype(np.int64)
    df[['source', 'target', 'edge']].to_csv(output_path, index=False)

In [13]:
def create_table(mode, length):
    create_table = f"""
        CREATE TABLE {mode}_{length}_path_table (
        id BIGSERIAL,
        source BIGINT,
        target BIGINT,
        edge BIGINT
        );
        """
        
    return create_table

In [14]:
def load_to_database(input_path, mode, length):
    copy_csv_query = f"COPY {mode}_{length}_path_table(source, target, edge) FROM STDIN DELIMITER ',' CSV HEADER"
    cursor.copy_expert(copy_csv_query, open(input_path, "r", encoding="utf-8"))

In [24]:
def create_final_table(mode, length):

    create_final_table = f"""
    CREATE TABLE {mode}_tmp AS
    SELECT DISTINCT
        {mode}_{length}_path_table.*,
        edge_table.x1,
        edge_table.y1
    FROM
        {mode}_{length}_path_table
    JOIN
        edge_table ON {mode}_{length}_path_table.source = edge_table.source;

    CREATE TABLE {mode}_{length} AS
    SELECT DISTINCT
        {mode}_tmp.*,
        edge_table.x2,
        edge_table.y2
    FROM
        {mode}_tmp
    JOIN
        edge_table ON {mode}_tmp.target = edge_table.target;

    DROP TABLE {mode}_tmp;
    DROP TABLE {mode}_{length}_path_table;


    ALTER TABLE {mode}_{length}
    ADD COLUMN line_geom geometry(LineString, 4326); 

    UPDATE {mode}_{length}
    SET line_geom = ST_SetSRID(ST_MakeLine(ST_SetSRID(ST_MakePoint(x1, y1), 4326), ST_SetSRID(ST_MakePoint(x2, y2), 4326)), 4326);

    """
    
    return create_final_table


In [16]:
def select_final_result(mode, length):
    return f"""SELECT st_union(line_geom) from {mode}_{length};"""

### undirected, short

In [19]:
mode = "undir"
length = "short"

original_input_path, transformed_input_path = set_paths(mode, length)

In [25]:
transform_csv_file(original_input_path, transformed_input_path)

cursor.execute(create_table(mode, length))
load_to_database(transformed_input_path, mode, length)
cursor.execute(create_final_table(mode, length))

### directed, short

In [26]:
mode = "dir"
length = "short"

original_input_path, transformed_input_path = set_paths(mode, length)

In [27]:
transform_csv_file(original_input_path, transformed_input_path)

cursor.execute(create_table(mode, length))
load_to_database(transformed_input_path, mode, length)
cursor.execute(create_final_table(mode, length))

### undirected, long

In [28]:
mode = "undir"
length = "long"

original_input_path, transformed_input_path = set_paths(mode, length)

In [29]:
transform_csv_file(original_input_path, transformed_input_path)

cursor.execute(create_table(mode, length))
load_to_database(transformed_input_path, mode, length)
cursor.execute(create_final_table(mode, length))

### directed, long

In [30]:
mode = "dir"
length = "long"

original_input_path, transformed_input_path = set_paths(mode, length)

In [31]:
transform_csv_file(original_input_path, transformed_input_path)

cursor.execute(create_table(mode, length))
load_to_database(transformed_input_path, mode, length)
cursor.execute(create_final_table(mode, length))